# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from utils.print_utils import printProgress, printWarning
from coastsat import NOC_tools, NOC_shoreline, NOC_download, NOC_preprocess,\
    SDS_tools, SDS_shoreline, SDS_download, SDS_preprocess

settings ={
           'output_epsg': 32620,
           'max_dist_ref': 100,           # max distance (in meters) allowed from the reference shoreline
           'min_length_shoreline': 500,   # min length (in meters) for shorelines
          }

# directory where the data will be accessed and stored
data_partition = 'c:\\data\\coastsat'
country = 'STV'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')
sites = os.listdir(sites_dir_path)

dates = [['2016-01-01', '2016-06-30'],
         ['2016-07-01', '2016-12-31'],
         ['2017-01-01', '2017-06-30'],
         ['2017-07-01', '2017-12-31'],
         ['2018-01-01', '2018-06-30'],
         ['2018-07-01', '2018-12-31'],
         ['2019-01-01', '2019-06-30'],
         ['2019-07-01', '2019-12-31'],
         ['2020-01-01', '2020-06-30'],
         ['2020-07-01', '2020-12-31']]

inputs = {'sat_name': 'S1', 'pixel_size': 10}

## download SAR data from GEE

In [ ]:
for site in sites:
    
    kml_filepath = os.path.join(sites_dir_path, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)

    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    inputs['polygon'] = roi_polygon
    inputs['site_name'] = site_name
    inputs['median_dir_path'] = median_dir_path
        
    for date_pair in dates:

        inputs['dates'] = date_pair 
        settings['inputs'] = inputs
        
        printProgress(f'processing {site_name}: {date_pair}')
        
        NOC_download.retrieve_median_sar(inputs)
        
    metadata = NOC_download.save_metadata(settings)

## find reference threshold and extract reference shoreline

In [ ]:
inputs['polarisation'] = 'VH'
inputs['dates'] = ['2016-01-01', '2020-12-31']
inputs['sigma'] = 1
    
for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    inputs['site_name'] = site_name
    inputs['median_dir_path'] = median_dir_path
    settings['inputs'] = inputs    
    
    printProgress(f'processing {site_name}')

    %matplotlib qt
    reference_threshold = NOC_shoreline.find_reference_threshold(settings)

## extract shorelines

In [ ]:
inputs['reference_threshold'] = reference_threshold
inputs['polarisation'] = 'VH'
settings['max_dist_ref'] = 50

for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    inputs['site_name'] = site_name
    inputs['median_dir_path'] = median_dir_path 
    settings['reference_shoreline'] = NOC_preprocess.load_reference_shoreline_median(inputs)
        
    for date_pair in dates:

        inputs['dates'] = date_pair
        settings['inputs'] = inputs    
        
        printProgress(f'processing {site_name}: {date_pair}')
        
        metadata = NOC_download.load_metadata(settings)

        %matplotlib qt
        NOC_shoreline.extract_shoreline_sar(metadata, settings)